# Ejericio Clustering
El objetivo es implementar un modelo que agrupa las transacciones apropiadamente y encontrar los potenciales outliers, es decir, aquellas transacciones que son sospechosas de ser un fraude o un error. Para resolver este ejercicio correctamente hay que investigar, en vez de simplemente seguir a rajatabla lo enseñado en el curso.

**Pistas:**

- Hemos explicado un algoritmo de clustering que no solo asigna elementos a clusters válidos, sino que también clasifica elementos como valores extremos (outliers). 


In [ ]:
import pandas as pd
df=pd.read_csv("./Datos/CC General.csv")

### Solución

In [ ]:
df.sample(10)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
customer_ids = df.CUST_ID
df = df.drop(columns="CUST_ID")

In [ ]:
df.columns[df.isnull().any()]

In [ ]:
df['MINIMUM_PAYMENTS'].hist();

In [ ]:
from sklearn.tree import DecisionTreeRegressor
X_train=df[~df["MINIMUM_PAYMENTS"].isna()].drop(columns=["CREDIT_LIMIT","MINIMUM_PAYMENTS"])
y_train=df[~df["MINIMUM_PAYMENTS"].isna()]["MINIMUM_PAYMENTS"]
X_test=df[df["MINIMUM_PAYMENTS"].isna()].drop(columns=["CREDIT_LIMIT","MINIMUM_PAYMENTS"])

In [ ]:
arbol=DecisionTreeRegressor()
arbol.fit(X_train,y_train)
y_predicho=arbol.predict(X_test)

In [ ]:
arbol.score(X_train,y_train)

In [ ]:
arbol.feature_importances_

In [ ]:
import matplotlib.pyplot as plt

feat_importances = pd.DataFrame(arbol.feature_importances_, index=X_train.columns, columns=["Importance"])
feat_importances.sort_values(by='Importance', ascending=False, inplace=True)
feat_importances.plot(kind='bar', figsize=(8,6))

In [ ]:
y_predicho

In [ ]:
df['CREDIT_LIMIT'].hist();

In [ ]:
df[df.isnull().any(axis=1)].shape

Imputamos a 0 los valores nulos, así podemos ver si los elementos anómalos son aquellos que tienen estas columnas a 0

In [ ]:
df = df.fillna(0)

Utilizo el algoritmo DBSCAN.
Y utilizo el StandardScaler para que todos los valores esten en la misma escala

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [ ]:
df_normalizado = pd.DataFrame(StandardScaler().fit_transform(df))

In [ ]:
clusterer = DBSCAN()
cluster_labels = clusterer.fit_predict(df_normalizado)
## muestro el top 5
pd.Series(cluster_labels).value_counts()[:5]

In [ ]:
pd.Series(cluster_labels).value_counts(normalize=True)[:5]

DBSCAN etiqueta practicamente un 75% de valores como -1. Usamos el coeficiente de silueta (`silhouette_score`) para ver como etiqueta el algoritmo

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
silhouette_score(df_normalizado, cluster_labels)

El valor es negativo, con lo que quiere decir que la segmetnación es bastante mala

In [ ]:
### Veamos los hiperparamentros que hemos utilizado
DBSCAN().get_params()

Vamos a realizar un similar a GridSearch para este modelo. ya que al no tener predict no podemos usar este o RandomSearch

In [ ]:
from scipy.stats import uniform,randint

distribucion_parametros = {
    "eps": uniform(0,5),
    "min_samples": randint(2, 20),
    "p": randint(1, 3),
}

In [ ]:
import numpy as np
from sklearn.model_selection import ParameterSampler

n_muestras = 30 # probamos 30 combinaciones de hiperparámetros
n_iteraciones = 2 #para validar, vamos a entrenar para cada 
                  # selección de hiperparámetros en 2 muestras distintas
pct_muestra = 0.7 # usamos el 70% de los datos para entrenar el modelo en cada iteracion
resultados_busqueda = []
lista_parametros = list(ParameterSampler(distribucion_parametros, n_iter=n_muestras))

for param in lista_parametros:
    for iteration in range(n_iteraciones):
        param_resultados = []
        muestra = df_normalizado.sample(frac=pct_muestra)
        etiquetas_clusters = DBSCAN(n_jobs=-1, **param).fit_predict(muestra)
        try:
            param_resultados.append(silhouette_score(muestra, etiquetas_clusters))
        except ValueError: # a veces silhouette_score falla en los casos en los que solo hay 1 cluster
            pass
    puntuacion_media = np.mean(param_resultados)
    resultados_busqueda.append([puntuacion_media, param])

In [ ]:
Parametros=sorted(resultados_busqueda, key=lambda x: x[0], reverse=True)[:5]

In [ ]:
Parametros

Cojo el primer valor que he obtenido

In [ ]:
mejores_params = Parametros[0][1]

clusterer = DBSCAN(n_jobs=-1, **mejores_params)

etiquetas_cluster = clusterer.fit_predict(df_normalizado)

In [ ]:
pd.Series(etiquetas_cluster).value_counts()

Vemos que hay 63 que se consideran anomalos

In [ ]:
def resumen_cluster(cluster_id):
    cluster = df[etiquetas_cluster==cluster_id]
    resumen_cluster = cluster.mean().to_dict()
    resumen_cluster["cluster_id"] = cluster_id
    return resumen_cluster

def comparar_clusters(*cluster_ids):
    resumenes = []
    for cluster_id in cluster_ids:
        resumenes.append(resumen_cluster(cluster_id))
    return pd.DataFrame(resumenes).set_index("cluster_id").T

In [ ]:
comparar_clusters(0,-1)